# Link Prediction Exercise

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

# Company Emails

The file 'email_prediction.txt' contains a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [14]:
# Build network G
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.


In [104]:
def salary_predictions():
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.metrics import roc_curve, auc
    
    # Convert network G into pandas dataframe
    df = pd.DataFrame(index=G.nodes())
    # Convert network node attributes into features
    df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(G.degree())
    
    feature= ['Department', 'clustering', 'degree']
    
    # Split data to train / test based on null values in management salary column
    # Train classifier on training set enables label predictions on the testing set
    X_test = df[df['ManagementSalary'].isnull()][feature]
    X_test = pd.DataFrame(X_test)
    X_train = df[~df['ManagementSalary'].isnull()][feature]
    X_train = pd.DataFrame(X_train)
    y_test = df[df['ManagementSalary'].isnull()]['ManagementSalary']
    y_train = df[~df['ManagementSalary'].isnull()]['ManagementSalary']
    
    # Use gradient Boosting classifier to train training set
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    clf = GradientBoostingClassifier(learning_rate = 0.01, max_depth = 2, random_state = 0).fit(X_train_scaled, y_train)
    # Output probability of each employee receiving management salary
    prediction = clf.predict_proba(X_test_scaled)
    result = pd.DataFrame(index=X_test.index, data = prediction)
    result = result.iloc[:, -1]
    result.name = None

    return result
salary_predictions()

1       0.170627
2       0.428395
5       0.638452
8       0.199105
14      0.170627
18      0.170627
27      0.170627
30      0.247082
31      0.247082
34      0.122151
37      0.199105
40      0.170627
45      0.157270
54      0.247082
55      0.199105
60      0.247082
62      0.638452
65      0.473540
77      0.100184
79      0.157270
97      0.102364
101     0.092544
103     0.247082
108     0.157270
113     0.401418
122     0.092544
141     0.170627
142     0.638452
144     0.102364
145     0.247082
          ...   
913     0.098059
914     0.100184
915     0.077132
918     0.122151
923     0.102364
926     0.102364
931     0.100184
934     0.077132
939     0.077132
944     0.077132
945     0.102364
947     0.102364
950     0.157270
951     0.102364
953     0.077132
959     0.077132
962     0.077132
963     0.192793
968     0.102364
969     0.102364
974     0.102364
984     0.077132
987     0.102364
989     0.102364
991     0.102364
992     0.077132
994     0.077132
996     0.0771

### New Connections Prediction

Predict future connections between employees of the network. The future connections information is included in the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [102]:
# Read data into pandas dataframe
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.


In [121]:
def new_connections_predictions():
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.metrics import roc_curve, auc
    
    # Use preferential attachment to predict probability of future connections between each employee pair
    df = pd.DataFrame(index=future_connections.index, data=future_connections['Future Connection'])
    df['prefered'] = [i[2] for i in nx.preferential_attachment(G, df.index)]
    
    # Find number of common neighbors between each employee pair in network G
    df['Common'] = df.index.map(lambda x: len(list(nx.common_neighbors(G, x[0], x[1]))))
    
    feature= ['prefered', 'Common']
    
    # Split data to train / test based on null values in future connection column
    X_test = df[df['Future Connection'].isnull()][feature]
    X_test = pd.DataFrame(X_test)
    X_train = df[~df['Future Connection'].isnull()][feature]
    X_train = pd.DataFrame(X_train)
    y_test = df[df['Future Connection'].isnull()]['Future Connection']
    y_train = df[~df['Future Connection'].isnull()]['Future Connection']
    
    # Use gradient Boosting classifier to train training set
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    clf = GradientBoostingClassifier(learning_rate = 0.01, max_depth = 2, random_state = 0).fit(X_train_scaled, y_train)
    
    # Output probability of future connections between each employee pair
    prediction = clf.predict_proba(X_test_scaled)
    result = pd.DataFrame(index=X_test.index, data = prediction)
    result = result.iloc[:, -1]
    result.name = None
    
    return result